### Imports

In [2]:
import os
import flywheel
import numpy as np
import pandas as pd
import nibabel as nib
from glob import glob
from os.path import join

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Custom helpers
import sys
sys.path.append("classes")
from helpers import *
from Model import Model
from Cloud import Cloud
from Button import Button
from Dicom import Dicom, DicomDir

### Load objects
cloud = Cloud()
# model = Model()

### Features

TODO:
- Working memory
- SCL-20, other behaviorial data

In [4]:
num_features = (
    2* (len(voxel_radius(5))) + # 4 radius voxels (previous and next volume)
#     (2* ((4 * 2) + 1)) + # trigger time for 4-radius (previous and next volume)
#     ((4 * 2) - 5) + # trigger time for current volume
    2 + # time since last gonogo stimuli
    2 + # time since go/nogo bloack started
    6 + # time since faces stimuli
    6 + # time since faces block started
    1 + # time since keypress (any)
    2 + # gender, age
    3 + # task identifier (is_gonogo, is_(non)conscious)
    3 + # study identifier (conn, engage, rad)
    0
)
"Number of features:", num_features

('Number of features:', 1053)

### Load settings

In [5]:
tasks  = "config/tasks.json"
EPOCHS = 100

### Helpers

Move these to own file when done

In [17]:
def _train_task(project, task, onsets, data, patient, is_ascending):
    for num_volume in len(data.shape[-1]):
        df = _gen_data(project, task, onsets, volume, patient, is_ascending, num_volume)
        model.train(df)

def _volume_data(data, x, y, z, t, direction, radius):
    row = dict()
    for coord in voxel_radius(radius):
        i, j, k  = coord["x"], coord["y"], coord["z"]
        loc      = "i%d_j%d_k%d_t%d" % (i, j, k, direction)
        row[loc] = data[x+i, y+j ,z+k, t+direction]
    return row

def _gen_avail_volumes(shape, radius):
    x_min, x_max = radius, shape[0] - radius
    y_min, y_max = radius, shape[1] - radius
    z_min, z_max = radius, shape[2] - radius
    return [(x, y, z) for x in range(x_min, x_max)
                      for y in range(y_min, y_max)
                      for z in range(z_min, z_max)
           ]
        
def _gen_data(project, task, onsets, dicoms, patient, is_ascending, num_volume, radius=5):
    avail_volumes = _gen_avail_volumes(dicoms.shape)
    for x, y, z in avail_volumes:
        _prev = _volume_data(dicoms, x, y, z, num_volume, -1, radius)
        _next = _volume_data(dicoms, x, y, z, num_volume, 1,  radius)
        row = {**_prev, **_next}
        row["signal"] = data[x, y, z, num_volume]   
    df = pd.DataFrame(rows)
    
    ### patient data
#     df["age"] = patient.age
#     df["is_female"] = (patient.gender == "female")
    
    ### Project data
    df["is_rad"] = (project == "rad")
    df["is_engage"] = (project == "engage")
    df["is_connectome"] = (project == "connectome")
    
    ### task data
    df["is_gng"] = (task == "gonogo")
    df["is_fc"]  = (task == "conscious")
    df["is_fnc"] = (task == "nonconscious")
    
    ### fMRI data
    df["ascending"] = int(is_ascending)
    return df

In [13]:
project = "connectome"
subject = "conn036"
time_session = "000_data_archive"
task = "conscious"
path = cloud.download(project, subject, time_session, task)

In [14]:
d = DicomDir(path)

unzipping /Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious.dicom.zip...


  0%|          | 7/6930 [00:00<01:49, 63.38it/s]

removing old path /Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious.dicom.zip...
loading dicoms from /Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious...
Sorting dicoms by trigger time...


100%|██████████| 6930/6930 [00:00<00:00, 275710.72it/s]


In [ ]:
onsets = None
patient = None
is_ascending = dicoms.is_ascending
num_volume = 3
_gen_data(project, task, onsets, dicoms, patient, is_ascending, num_volume)

### Build model

In [19]:
def build_model(num_features):
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=(num_features, 1)),
        layers.Conv1D(100, 5, activation=tf.nn.relu), ### FIXME: check the filters and kernel_size
        layers.Dense(64, activation=tf.nn.relu),
        layers.AveragePooling1D(pool_size=2),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam()

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

model = build_model(num_features)

In [ ]:
### Iterate over tasks
for task in tasks:
    task_info = tasks[task]
    num_vols  = task_info["vol"]
    if "project" in task_info:
        projects = task_info["project"]
    else:
        projects = ["rad", "connectome", "engage"]
        
    ### Iterate over projects
    for project in projects:
        time_sessions = project_config["time_sessions"]
        subjects = load_project_subjects(project)
        
        ### Iterate over time_session/subject
        for time_session in time_sessions:
            for subject in subjects:
                ### Set logger info
                logger.set(project=project, subject=subject, time_session=time_session, task=task)
            
                ### Load necessary data
                patient    = Patient(project, subject, time_session)
                dicom_path = cloud.download(project, subject, time_session, task)
                button     = Button(project, subject, time_session, task)
                
                ### Check all data exists
                if not _valid_patient(patient, dicom_path, button, logger):
                    continue
                    
                ### More setup
                fmri_data = DicomDir(dicom_path)
                fmri_data.cut_volumes(fmri_data.num_volumes - num_vols)
                
                ### Modeling
                model.load(model_path)
                model.run(patient, fmri_data, button)
                model.save(model_path)